In [94]:
from osgeo import gdal, osr, ogr
import os
BAND_DESCRIPTION = {'B1': "Coastal-Aerosol",
                    'B2': "Blue",
                    'B3': "Green",
                    'B4': "Red",
                    'B5': "NIR",
                    'B6': "SWIR-1",
                    'B7': "SWIR-2",
                    'B8': "PAN",
                    'B9': "Cirrus"}

# Cropping a geotiff in python
*This is included not as an excercise but as optional exposure to the data processign methods used in preperation for the raster exercise.  This relies on files not provided as a space saving measure, but can be adapted to any landsat tile.*


We have some landsat data we want to use in an exercise.  However, the landsat tile we got from earth explorer is very large, over 80 megabytes per image band!  We are using this data for an example, so we want just a subset around boulder.  Lets crop to just that using Python.  We will use a Python library called GDAL (Geospatial Data Abstraction Library) to help us crop the image to just the location we want.

First lets open one of the rasters (Landsat Band 2) in gdal:

In [67]:
band_2=gdal.Open("LC08_L2SP_034032_20230711_20230718_02_T1_SR_B2.TIF")

Now that we have the raster loaded as a python object, we can see it's spatial projection:

In [68]:
raster.GetProjection()

'PROJCS["WGS 84 / UTM zone 13N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32613"]]'

There's a lot of information here that may or may not make sense to you depending on how much GIS you've done, but the key part is "WGS 84 / UTM zone 13N".  That tells us the coordiante system we'll need to provide our coordinates in for our bounding box that we use to crop.  I went to [OpenStreetMap](https://www.openstreetmap.org) to get the coordinates I want to use to crop, and it gave them to me in latitude and longitude.

In [69]:
top_left_lat_long = (40.1028,-105.4097)
bottom_right_lat_long = (39.9424, -105.1907)

While great for humans, this won't match up with our raster, which is in UTM zone 13.  If we look at the bounding coordinates for our raster, we'll see the numbers totally don't make sense for latitude and longitude.  We can see this information with the GetGeoTransform() function on the raster, which returns (in order):
1. The x-coordinate of the upper-left croner of the upper left pixel
2. The pixel width (in the units of the coordinate system)
3. The row rotation (don't worry about it)
4. the y-coordinate of the upper left corner of the upper left pixel
5. The column rotation (again, don't worry about it)
6. The pixel height (in the units of the coordinate system)

In [70]:
band_2.GetGeoTransform()

(316785.0, 30.0, 0.0, 4583115.0, 0.0, -30.0)

This tells us the the upper left of the image is located at point (316785, 4583115) in the UTM zone 13N grid, each pixel is 30 by 30 meters.
Now we'll use some GDAL functions to transform the point from latitude/longitude to UTM zone 13.  This is in someways overkill (we could just find an online converter and use that) as it will involve making a geometry object and adding our coordinates, but it will expose us to some GDAL functions.  First we'll make a multi point geometry object

In [71]:
points = ogr.Geometry(ogr.wkbMultiPoint)

Our starting points are geographic Coordiantes in the WGS84 datum.  This has a well known ID number of 4326.  We'll create a spatial reference object with that coordiante system, and set that to our new geometry object

In [72]:
input_spatial_reference = osr.SpatialReference()

In [73]:
input_spatial_reference.ImportFromEPSG(4326)

0

In [74]:
points.AssignSpatialReference(input_spatial_reference)

Now let's create point geometry objects of our bounding points to add to our multipoint object

In [75]:
top_left = ogr.Geometry(ogr.wkbPoint)
top_left.AddPoint(*top_left_lat_long)
bottom_right = ogr.Geometry(ogr.wkbPoint)
bottom_right.AddPoint(*bottom_right_lat_long)

Woah!  What's that * doing there?  The AddPoint function takes 2 arguments, an x coordiante and a y coordinate.  However, top_left_lat_long is a tuple.  If I were to call the function like this

In [76]:
top_left.AddPoint(top_left_lat_long)

TypeError: Geometry_AddPoint() missing required argument 'y' (pos 3)

We get an error that I'm missing the y argument.  What's happening is that it tries to set the whole tuple as the x coordiante and nothing as the y coordinate.  I could have called the function like this:

In [77]:
top_left.AddPoint(top_left_lat_long[0], top_left_lat_long[1])

but instead I prefaced the tuple with an astirix to indicate that the tuple should be "unpacked" with the first value set as the first argument, and the second as the second.  Don't worry about this, but it's something you can do, and a feature that I use, so I wanted to explain it.

Now we have our points objects, we can add them to our multipoint object

In [78]:
points.AddGeometry(top_left)
points.AddGeometry(bottom_right)

0

Our individual point objects are safely stored away in our points object, an can be recalled like so:

In [79]:
points.GetGeometryRef(0)

<osgeo.ogr.Geometry; proxy of <Swig Object of type 'OGRGeometryShadow *' at 0x7f3ed8d1fd50> >

That will come in handy to find the coordinates after we transform them to the new projection.
Next we need to create a spatial reference object that matches the projection of our raster.

In [80]:
raster_spatial_reference = osr.SpatialReference(
    wkt=raster.GetProjection())

Now we can transform our points to the new spatial reference

In [81]:
points.TransformTo(raster_spatial_reference)

0

And now we can pull out our transformed points, and view their coordinates

In [88]:
top_left_transformed = points.GetGeometryRef(0)
bottom_right_transformed = points.GetGeometryRef(1)
top_left_utm = (top_left_transformed.GetX(),
                top_left_transformed.GetY())
bottom_right_utm = (bottom_right_transformed.GetX(),
                    bottom_right_transformed.GetY())
print(top_left_utm)
print(bottom_right_utm)

(465080.5794855079, 4439247.537483415)
(483708.23085748643, 4421381.6205978)


Now we have everything we need to create our window for cropping

In [89]:
window = (top_left_transformed.GetX(),
          top_left_transformed.GetY(),
          bottom_right_transformed.GetX(),
          bottom_right_transformed.GetY())

We will loop over the tiff files in the current directory, renaming and cropping as we go:

In [98]:
for file in os.listdir('.'):
    name, extension = os.path.splitext(file)
    if extension == ".TIF":
        band = name.split('_')[-1]
        new_file_name = "sample_landsat_%s.TIF" % BAND_DESCRIPTION[band]
        gdal.Translate(new_file_name, file, projWin= window)